# Minimum wages around the world

Jamaica recently increased its mimumum wage. Out of curiosity, let's see what Wikipedia says about how other countries' minimum wages stack up.

First we import some tools to get our data .

In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas import DataFrame as df

Then we scrape from Wikipedia.

In [12]:
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage")
soup = BeautifulSoup(webpage.text, 'html.parser')
soup.prettify()
table = soup.find_all('table', class_ = "wikitable sortable static-row-numbers plainrowheaders srn-white-background")

Now we have the data, time to pick out each country and its hourly PPP mimimum wage.

In [81]:
tableRows = [[td.text.strip('\n') for td in row.find_all("td")] for row in table.find_all("tr")[1:]]
tableHeaders = [th.text.strip('\n') for th in table.find_all("th")]
tableHeaders.pop(2)
tableHeaders.pop(3)
data = df(tableRows, columns=tableHeaders)
wages = data[['Country','PPP (Int$)[7]']]
wages['PPP (Int$)[7]'].replace('', np.nan, inplace=True)
wages = wages.dropna()
wages = wages[pd.to_numeric(wages['PPP (Int$)[7]'], errors='coerce').notnull()]
wages.to_csv('intl_min_wages.csv')

In [47]:
!pip install country-converter

  Created wheel for country-converter: filename=country_converter-0.7.4-py3-none-any.whl size=53543 sha256=c7986dd4e9126960bf76f10ad192fe8b81786504141649517e3710600320da3a
  Stored in directory: c:\users\jason\appdata\local\pip\cache\wheels\fa\46\93\aa5e6b53178cc2356f30207643e869b03b7bc83e78a1b589bf
Successfully built country-converter


In [96]:
import plotly.graph_objects as go
import country_converter as coco
cc = coco.CountryConverter()

fig = go.Figure(data=go.Choropleth(
    locations = cc.convert(names=wages['Country'], to='ISO3'),
    z = wages['PPP (Int$)[7]'],
    text = wages['Country'],
    colorscale = 'Greens',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'Minimum hourly wage (PPP$)',
))

fig.update_layout(
    title_text='Minimum wages (PPP$) per hour worldwide',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text= "",#'Source: <a href="https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage#List">\
       #     Wikipedia</a>',
        showarrow = False
    )]
)

fig.show()

 Republic of the Congo not found in regex
 Samoa not found in regex


Argentina is clearly an outlier. They've had a weird situation economically for the past century. Let's see how this maps to unemployment.

We'll use the average unemployment rate for the past 5 years from the World Bank's dataset at https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS

In [91]:
ud = pd.read_csv("API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_3629004.csv")
ud['avg5'] = sum([ud['2020'],ud['2019'],ud['2018'],ud['2017'],ud['2016']])/5
unemployment = ud[['Country Name','Country Code','avg5']]
unemployment = unemployment.dropna()
unemployment

,Country Name,Country Code,avg5
1,Africa Eastern and Southern,AFE,6.906773
2,Afghanistan,AFG,11.283400
3,Africa Western and Central,AFW,6.093178
4,Angola,AGO,7.599000
5,Albania,ALB,13.227800
...,...,...,...
260,Samoa,WSM,8.625200
262,"Yemen, Rep.",YEM,13.264400
263,South Africa,ZAF,27.636000
264,Zambia,ZMB,11.976000


In [98]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = go.Figure(data=go.Choropleth(
    locations = cc.convert(names=unemployment['Country Name'], to='ISO3'),
    z = unemployment['avg5'],
    text = unemployment['Country Name'],
    colorscale = 'Reds',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    # colorbar_tickprefix = '$',
    colorbar_title = 'Unemployment rate (%)',
))

fig.update_layout(
    title_text='Unemployment rates worldwide',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='',#'Source: <a href="https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS">\
           # World Bank</a>',
        showarrow = False
    )]
)

fig.show()

Africa Eastern and Southern not found in regex
Africa Western and Central not found in regex
Arab World not found in regex
Central Europe and the Baltics not found in regex
Channel Islands not found in regex
Caribbean small states not found in regex
East Asia & Pacific ( not found in regex
Early-demographic dividend not found in regex
East Asia & Pacific not found in regex
Europe & Central Asia ( not found in regex
Europe & Central Asia not found in regex
Euro area not found in regex
European Union not found in regex
Fragile and conflict affected situations not found in regex
High income not found in regex
Heavily indebted poor countries (HIPC) not found in regex
IBRD only not found in regex
IDA & IBRD total not found in regex
IDA total not found in regex
IDA blend not found in regex
IDA only not found in regex
Latin America & Caribbean ( not found in regex
Latin America & Caribbean not found in regex
Least developed countries: UN classification not found in regex
Low income not found 

,Country,PPP (Int$)[7],Country Name,Country Code,avg5
1,Afghanistan,1.57,NaN,NaN,NaN
2,Albania,3.14,NaN,NaN,NaN
3,Algeria,3,NaN,NaN,NaN
4,Andorra,5,NaN,NaN,NaN
5,Angola,1.38,NaN,NaN,NaN
...,...,...,...,...,...
260,NaN,NaN,Samoa,WSM,8.6252
262,NaN,NaN,"Yemen, Rep.",YEM,13.2644
263,NaN,NaN,South Africa,ZAF,27.6360
264,NaN,NaN,Zambia,ZMB,11.9760
